<a href="https://colab.research.google.com/github/mrchapagain/ConsumerDataAnalytics/blob/main/Big_Food_Database.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pips install
#!pip install evidently
#!pip install facets-overview
#!pip install yfinance
#!pip install itables # for interactive dataframe display

In [1]:
import  pandas as pd

#libaries for display and interactivity
from itables import init_notebook_mode
init_notebook_mode(all_interactive=True)

from IPython.core.display import display, HTML
from google.colab import output
#output.enable_custom_widget_managet()

from ipywidgets import interact, interactive
import ipywidgets as widgets

from evidently.dashboard import Dashboard
from evidently.dashboard.tabs import DataDriftTab

from itables import init_notebook_mode, show
init_notebook_mode(all_interactive=True)

import warnings
warnings.filterwarnings("ignore")

#Open Data-set on Food-Nutrition-health from research sector

***FRIDA Food Composition Database: ***

*   ***Category list:*** https://frida.fooddata.dk/food/lists/grouped?lang=en
*   ***Data display inspiration:*** https://foodb.ca/foods/FOOD00011
*   ***Display on FRIDA:*** https://frida.fooddata.dk/food/754?lang=en

In [2]:
# Import FRIDA NNutrient data
path_frida_data="https://github.com/mrchapagain/ConsumerDataAnalytics/raw/main/FridaNutrientData.xlsx"

# Lets open data as they are but only 26 columns out of 201 columns
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=0, skiprows=0).round(decimals = 2).iloc[:, :25]
data_frida.head(3)

Unnamed: 1         Unnamed: 2 Svind Energi, kJ Energi, kcal  \
Nummer      gruppe               Navn     %         kJ         kcal   
1         Bærfrugt        Jordbær, rå     4        162           38   
2       Kernefrugt  Æble, uspec., råt    10        233           55   

       Nitrogen-til-protein faktor Nitrogen, total Protein, videnskabelig  \
Nummer                           -               g                      g   
1                             6.25           0.106                    0.7   
2                             6.25           0.043                    0.3   

       Protein, deklaration Kulhydrat, difference   ...  FCF Alkohol Aske  \
Nummer                    g                      g  ...    -       g    g   
1                       0.7                    8.3  ...  0.8       0  0.4   
2                       0.3                   14.3  ...  0.8       0  0.3   

       Tørstof  Vand A-vitamin Retinol beta-caroten D-vitamin D3-vitamin  
Nummer       g     g        RE      µg           µg        µg         µg  
1           10    90      3.33       0           40         0          0  
2         15.1  84.9      2.08       0           25         0          0  

[3 rows x 25 columns]

In [3]:
# Lets define columns name that are readeable
column_names= ['Gruppe', 'Navn', 'Svind_%', 'Energy_kj', 'Energy_kcal', 'Nitrogen_til_protein_faktor', 'Nitrogen_total_g', 'Protein_videnskabelign_g', 'Protein_deklaration_g', 'Kulhydrat_difference_g', 'Kulhydrat_tilgængelig_g', 
               'Kulhydrat_deklaration_g', 'Tilsat_sukker_g', 'Kostfibre_g', 'Fedt_total_g', 'FCF', 'Alkohol_g', 'Aske_g', 'Tørstof_g', 'Vand_g', 'A_vitamin_RE', 'Retinol_µg', 'beta_caroten_µg', 'D_vitamin_µg', 'D3_vitamin_µg']

# Lets open with little modificaton so easy to read and understand
data_frida= pd.read_excel(path_frida_data, sheet_name=1, index_col=0, header=None, skiprows=2).round(decimals = 2).iloc[:, :25].reset_index(drop=True)

# Assign columns name that defined just above
data_frida.columns= column_names

# lets see somerowsof the data
data_frida.head(3)

Gruppe               Navn Svind_% Energy_kj Energy_kcal  \
0    Bærfrugt        Jordbær, rå       4       162          38   
1  Kernefrugt  Æble, uspec., råt      10       233          55   
2    Sydfrugt          Banan, rå      41       396          93   

  Nitrogen_til_protein_faktor Nitrogen_total_g Protein_videnskabelign_g  \
0                        6.25            0.106                      0.7   
1                        6.25            0.043                      0.3   
2                        6.25            0.183                      1.1   

  Protein_deklaration_g Kulhydrat_difference_g  ...   FCF Alkohol_g Aske_g  \
0                   0.7                    8.3  ...   0.8         0    0.4   
1                   0.3                   14.3  ...   0.8         0    0.3   
2                   1.1                   22.6  ...  0.73         0    0.7   

  Tørstof_g Vand_g A_vitamin_RE Retinol_µg beta_caroten_µg D_vitamin_µg  \
0        10     90         3.33          0              40            0   
1      15.1   84.9         2.08          0              25            0   
2      24.6   75.3         4.42          0              53            0   

  D3_vitamin_µg  
0             0  
1             0  
2             0  

[3 rows x 25 columns]

In [7]:
row=data_frida[data_frida.Navn== "Kylling, bryst (filet), kogt, pålæg"]
print("Id: ", row.index[0])
print("Category: ", row.Gruppe.values[0])
print("Name: ", row.Navn.values[0])
print("Total Energy: ", row.Energy_kj.values[0], "Kj (", row.Energy_kcal.values[0], "Kacl)")
print("Total Energy: ", row.Energy_kcal.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])
print("Total Energy: ", row.Energy_kj.values[0])

Id:  561
Category:  Kylling og høne
Name:  Kylling, bryst (filet), kogt, pålæg
Total Energy:  475 Kj ( 113 Kacl)
Total Energy:  113
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475
Total Energy:  475


**FlavourDB: Database of flavor molecules**
*   link ="https://www.ars.usda.gov/northeast-area/beltsville-md-bhnrc/beltsville-human-nutrition-research-center/methods-and-application-of-food-composition-laboratory/mafcl-site-pages/flavonoids/"
*   Academic Article= https://academic.oup.com/nar/article/46/D1/D1210/4559748
*   *FlavourDB= https://cosylab.iiitd.edu.in/flavordb/*

***Food Database called "FoodDB"***

for inspiiration
Link= https://foodb.ca/foods/FOOD00008


In [ ]:
from google.colab import files
uploaded = files.upload()

Saving OntologyTerm.json to OntologyTerm.json


In [ ]:
type(uploaded)
# # Get the list of all files and directories with JSON files
food_db_dir= "/content/FoodDB_JSON.zip" #"C:/Users/Bruger/Desktop/dataset/FoodDB_JSON/" # The directory with the JSON files

dict

In [ ]:
#Food Database called "FoodDB"
import os
import json
import pandas as pd
import urllib
import zipfile

file_url_link= "https://fooddb.ca/public/system/downloads/foodb_2020_04_07_json.zip"
file_name = 'food_db.zip'

urllib.request.urlretrieve(file_url_link, file_name)
os.listdir()

## Lets extract the zipped file first
zip_ref = zipfile.ZipFile("food_db.zip", "r")
zip_ref.extractall()
zip_ref.close()

In [4]:
## Lets open one of the extracted file
file_link="/content/foodb_2020_04_07_json/Content.json"

chunks = pd.read_json(file_link, lines=True, chunksize = 5)
for chunk in chunks:
    show(chunk, maxBytes=0)
    break

In [5]:
file_link="/content/foodb_2020_04_07_json/Food.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 5855)
for chunk in chunks:
    a= chunk[chunk.name== "Grape"]
    display(a[["id", "name", "name_scientific", "ncbi_taxonomy_id", "public_id"]])
    break

id   name name_scientific  ncbi_taxonomy_id  public_id
368  374  Grape           Vitis            3603.0  FOOD00369

In [65]:
file_link="/content/foodb_2020_04_07_json/Content.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 55555)
for chunk in chunks:
    a= chunk[chunk.food_id== 374]
    display(a[["id", "source_id", "source_type", "food_id", "orig_food_id", "orig_food_common_name", "orig_food_scientific_name"]])
    break

id  source_id source_type  food_id  orig_food_id  \
3435    3795          2    Nutrient      374          1087   
3436    3796          2    Nutrient      374          1087   
3437    3797          2    Nutrient      374           329   
3438    3798          2    Nutrient      374           329   
3439    3799          2    Nutrient      374           328   
...      ...        ...         ...      ...           ...   
51087  56031         15    Nutrient      374          9136   
51088  56032         15    Nutrient      374          9137   
51089  56033         15    Nutrient      374         11975   
51090  56034         15    Nutrient      374         14277   
51091  56035         15    Nutrient      374         14282   

                                   orig_food_common_name  \
3435                                         Grape juice   
3436                                         Grape juice   
3437                                          Grape, raw   
3438                                          Grape, raw   
3439                                      Oil, grapeseed   
...                                                  ...   
51087  Grape juice cocktail, frozen concentrate, undi...   
51088  Grape juice cocktail, frozen concentrate, dilu...   
51089                               Grape leaves, canned   
51090                                Grape drink, canned   
51091                          Grape juice drink, canned   

      orig_food_scientific_name  
3435                       None  
3436                       None  
3437                       None  
3438                       None  
3439                       None  
...                         ...  
51087                      None  
51088                      None  
51089                      None  
51090                      None  
51091                      None  

[94 rows x 7 columns]

In [34]:
file_link="/content/foodb_2020_04_07_json/Nutrient.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 55555)
for chunk in chunks:
    a= chunk[chunk.legacy_id== 374]
    display(a[["id", "legacy_id", "name", "public_id"]])
    break

id  legacy_id                        name  public_id
0    1    10930.0                         Fat  FDBN00001
1    2    10946.0                    Proteins  FDBN00002
2    3    16037.0                Carbohydrate  FDBN00003
3    4    23404.0                 Fatty acids  FDBN00004
4    5    11134.0             Fiber (dietary)  FDBN00005
5    6    58893.0                        13:0  FDBN00006
6    7    58894.0                        14:1  FDBN00007
7    8    58895.0                        15:1  FDBN00008
8    9    58896.0                      16:1 c  FDBN00009
9   10    58897.0                      16:1 t  FDBN00010
10  11    58898.0       16:1 undifferentiated  FDBN00011
11  12    58899.0                        17:1  FDBN00012
12  13    58900.0                      18:1 c  FDBN00013
13  14    58901.0                      18:1 t  FDBN00014
14  15    58902.0       18:1 undifferentiated  FDBN00015
15  16    58903.0                   18:2 CLAs  FDBN00016
16  17    58904.0                      18:2 i  FDBN00017
17  18    58905.0  18:2 t not further defined  FDBN00018
18  19    58906.0                    18:2 t,t  FDBN00019
19  20    58907.0       18:2 undifferentiated  FDBN00020
20  21    58908.0       18:3 undifferentiated  FDBN00021
21  22    58909.0                       18:3i  FDBN00022
22  23    58910.0                        18:4  FDBN00023
23  24    58911.0                        20:1  FDBN00024
24  25    58912.0                20:2 n-6 c,c  FDBN00025
25  26    58913.0                    20:3 n-3  FDBN00026
26  27    58914.0                    20:3 n-6  FDBN00027
27  28    58915.0       20:3 undifferentiated  FDBN00028
28  29    58916.0       20:4 undifferentiated  FDBN00029
29  30    58917.0                        21:5  FDBN00030
30  31    58918.0                        22:0  FDBN00031
31  32    58919.0                      22:1 c  FDBN00032
32  33    58920.0                      22:1 t  FDBN00033
33  34    58921.0       22:1 undifferentiated  FDBN00034
34  35    58922.0                        22:4  FDBN00035
35  36    58923.0                    22:5 n-3  FDBN00036
36  37    58924.0                      24:1 c  FDBN00037
37  38    58928.0                      Energy  FDBN00038
38  39        NaN                         Ash  FDBN00039

In [49]:
file_link="/content/foodb_2020_04_07_json/Compound.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 55555)
for chunk in chunks:
    a= chunk[chunk.public_id== "FDB000004"]
    display(a[["id", "public_id", "name", "description"]])#, "cas_number", "kingdom", "subklass", "klass", "subklass
    break

id  public_id                                 name  \
0   4  FDB000004  Cyanidin 3-(6''-acetyl-galactoside)   

                                         description  
0  Constituent of the leaves of Nymphaea alba [CC...

In [63]:
file_link="/content/foodb_2020_04_07_json/Flavor.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 558555)
for chunk in chunks:
    a= chunk[chunk.name== "grape"]
    display(a[["id", "name", "flavor_group"]])
    break

id   name flavor_group
76  77  grape       fruity

In [75]:
file_link="/content/foodb_2020_04_07_json/CompoundsFlavor.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 55555)
for chunk in chunks:
    a= chunk#[chunk.name== "grape"]
    display(a[["id", "compound_id", "flavor_id", "source_id", "source_type"]])
    break

id  compound_id  flavor_id  source_id source_type
0          1        11947        159      11947    Compound
1          2         8298        213       8298    Compound
2          3         8298        156       8298    Compound
3          4        11889        231      11889    Compound
4          5        11889        166      11889    Compound
...      ...          ...        ...        ...         ...
11770  11798        10785        209      10785    Compound
11771  11799        10785        229      10785    Compound
11772  11800        10542        883      10542    Compound
11773  11801        31233        157      31233    Compound
11774  11802         5218        114       5218    Compound

[11775 rows x 5 columns]

In [77]:
file_link="/content/foodb_2020_04_07_json/FoodTaxonomy.json"

chunks = pd.read_json(file_link, lines=True, chunksize= 558555)
for chunk in chunks:
    a= chunk#[chunk.name== "grape"]
    display(a[["id", "food_id", "ncbi_taxonomy_id", "classification_name"]])
    break

id  food_id  ncbi_taxonomy_id classification_name
0      1        1            357850         "Eukaryota"
1      2        1            357850     "Viridiplantae"
2      3        1            357850      "Streptophyta"
3      4        1            357850       "Embryophyta"
4      5        1            357850      "Tracheophyta"
..   ...      ...               ...                 ...
884  915      909              4072         "Solanales"
885  916      909              4072        "Solanaceae"
886  917      909              4072       "Solanoideae"
887  918      909              4072         "Capsiceae"
888  919      909              4072          "Capsicum"

[889 rows x 4 columns]

***FoodOn: A farm to fork ontology***

*   link= https://foodon.org/

> Application of FoodOn: https://foodon.org/wp-content/uploads/2020/11/image-768x310.png 

*   FoodKG: A Semantics-Driven Knowledge Graph for Food Recommendation:https://foodkg.github.io/subs.html

In [ ]:
# FoodOn:A harmonized food ontology to increase global food traceability


***FoodOntomap:* Linking food concepts across Different Food Ontologies**
Link of Database: https://zenodo.org/record/3600619/files/FoodOntoMap_v2.zip?download=1


***Food Image datasets:***

*   **link:** https://www.epfl.ch/labs/mmspg/downloads/food-image-datasets/
*   **Food-5K:** Dataset of food-imgesand non-food images, resulting in a totoal of 5,000 images
*   **Food-11:** dataset consist of 16,643 images grouped into 11 categories, which basiically covers the major types of food that people consumer in daily life

*** FAO Food Composition Databases ***
*   Link: https://www.fao.org/infoods/infoods/tables-and-databases/faoinfoods-databases/en/
*   List item



***NutriChem 2.0:* exploring the effect of plant-based foods on human health and drug efficacy**

***DietHub:* Dietary habits analysis through understanding the content of recipes**

***US food Data central ***
API key= https://api.nal.usda.gov/fdc/v1/foods/search?query=apple&pageSize=2&api_key=JdC6qVfCHr3AEEaiKkQrUFZGrXncNUGup7E5crN1